## language model using a bert model
* take 1

* Try 1 : keras based [bert-for-tf2], https://github.com/kpe/bert-for-tf2 

In [2]:
# !pip install sentencepiece
# ! pip install pytorch_transformers

  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp37-none-any.whl size=884004 sha256=649cc9b2ce307eb2f82f4ebe05e7fd115a57f9d5793cdc9fcecea4fb70dec4f5
  Stored in directory: C:\Users\Dan Ofer\AppData\Local\pip\Cache\wheels\63\2a\db\63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses


In [1]:
# import subprocess
# import keras

import numpy as np
import pandas as pd

import sentencepiece as spm

%matplotlib inline

In [2]:
sequences_file = "./data/sample_seqs.csv"

In [4]:
proteins = pd.read_csv('data/proteins.csv.gz')
proteins["seq_len"] = proteins["seq"].str.len()
print(proteins.shape)
proteins.head()

(1000000, 6)


,tax_id,uniprot_name,seq,n_complete_go_annotations,complete_go_annotation_indices,seq_len
0,1572043.0,A0A5A9P0L4_9TELE,MEEITQIKKRLSQTVRLEGKEDLLSKKDSITNLKTEEHVSVKKMVI...,0,[],45354
1,1930593.0,A0A410P257_9BACT,MKAIAWLIVLTFLPEQVAWAVDYNLRGALHGAVAPLVSAATVATDG...,0,[],39677
2,72004.0,A0A5A9Q4C1_9CETA,MSSQESPAVEFSTTTVSSVAVQAGDSKIVIAVIKCGKWIQLQLAES...,0,[],38354
3,375764.0,UPI0011814A49,MSTQAPSFTQPLQSVVALEGSAATFQAQVSGSPVPEVSWFRDGQVL...,0,[],38182
4,1415580.0,UPI001186A1A9,MTTEAPKFLQPLQSVVALEGSAATFEARISGFPVPEVSWFRDGQVL...,0,[],38163


In [8]:
proteins.head()

,tax_id,uniprot_name,seq,n_complete_go_annotations,complete_go_annotation_indices,seq_len
0,1572043.0,A0A5A9P0L4_9TELE,MEEITQIKKRLSQTVRLEGKEDLLSKKDSITNLKTEEHVSVKKMVI...,0,[],45354
1,1930593.0,A0A410P257_9BACT,MKAIAWLIVLTFLPEQVAWAVDYNLRGALHGAVAPLVSAATVATDG...,0,[],39677
2,72004.0,A0A5A9Q4C1_9CETA,MSSQESPAVEFSTTTVSSVAVQAGDSKIVIAVIKCGKWIQLQLAES...,0,[],38354
3,375764.0,UPI0011814A49,MSTQAPSFTQPLQSVVALEGSAATFQAQVSGSPVPEVSWFRDGQVL...,0,[],38182
4,1415580.0,UPI001186A1A9,MTTEAPKFLQPLQSVVALEGSAATFEARISGFPVPEVSWFRDGQVL...,0,[],38163


In [9]:
proteins.describe()

,tax_id,n_complete_go_annotations,seq_len
count,9.999770e+05,1000000.000000,1000000.000000
mean,5.562390e+05,9.067817,2336.231343
std,7.443289e+05,17.448869,1427.527815
min,2.000000e+00,0.000000,11.000000
25%,1.014100e+04,0.000000,1687.000000
50%,1.258780e+05,0.000000,1958.000000
75%,1.037660e+06,12.000000,2486.000000
max,2.651628e+06,632.000000,45354.000000


In [10]:
proteins.loc[proteins["seq_len"]<4000].shape[0]

(928988, 6)

### output sample of sequences for LM training
* We run the sentence piece on this beforehand in the other notebok - sentencepiece 
* http://localhost:8888/notebooks/toyData_ProBERT.ipynb

In [12]:
proteins.loc[proteins["seq_len"]<5000].sample(n=30000)["seq"].to_csv("./data/sample_seqs.csv", index = False)

C:\Users\Dan Ofer\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [ ]:
# # spm.SentencePieceTrainer.Train('--input=uniclust30_2016_1M_sampledSeq.fasta --model_prefix=m --vocab_size=1500')
# spm.SentencePieceTrainer.Train('--input=%s --model_prefix=m --vocab_size=16000'%sequences_file)

## tokenize using opreviuously trained sentence pioece

In [5]:
# makes segmenter instance and loads the model file (m.model)
sp = spm.SentencePieceProcessor()
sp.load('m.model')

True

In [6]:
eg_seq = "MRYTVLIALQGALLLLLLIDDGQGQSPYPYPGMPCNSSRQCGLGTCVHSRCAHCSSDGTLCSPEDPTMVWPCCPESSCQLVVGLPSLVNHYNCLPNQCTDSSQCPGGFGCMTRRSKCELCKADGEACNSPYLDWRKDKECCSGYCHTEARGLEGVCIDPKKIFCTPKNPWQLAPYPPSYHQPTTLRPPTSLYDSWLMSGFLVKSTTAPSTQEEEDDY"
print(sp.encode_as_pieces(eg_seq))
print(sp.encode_as_ids(eg_seq))

['▁', 'MR', 'YTV', 'LIA', 'L', 'QGA', 'L', 'LLLLL', 'IDD', 'GQGQ', 'SPY', 'PYP', 'GMP', 'CNS', 'SRQ', 'CGL', 'GTC', 'VHS', 'RCA', 'HCS', 'SDGT', 'LCS', 'PED', 'PT', 'MV', 'WP', 'CCP', 'ES', 'SCQ', 'L', 'VVG', 'LPSL', 'VNH', 'YNC', 'LPN', 'QCT', 'DSS', 'QCP', 'GGFG', 'CM', 'TRR', 'SK', 'CELC', 'KAD', 'GEA', 'CNS', 'PYL', 'DWR', 'KDKE', 'CCS', 'GY', 'CH', 'TEA', 'RGLE', 'GVC', 'IDP', 'KKI', 'FCT', 'PKNP', 'WQ', 'LAP', 'YPP', 'SYH', 'QPT', 'TLR', 'PPTS', 'LYD', 'SWL', 'M', 'SGF', 'LVK', 'STTA', 'PST', 'QEEE', 'DDY']
[0, 261, 3435, 2112, 3, 2866, 3, 7330, 1389, 9500, 3091, 4060, 5365, 6116, 2653, 5790, 5014, 3522, 6044, 6251, 7308, 3364, 1347, 165, 227, 606, 6383, 76, 5146, 3, 917, 8768, 6529, 11718, 2114, 8040, 498, 8400, 8666, 683, 2705, 70, 11787, 3365, 1052, 6116, 3694, 6163, 8661, 6444, 255, 503, 1356, 14930, 3704, 2768, 1160, 9902, 15766, 596, 1937, 6559, 6055, 2673, 989, 7826, 3396, 4156, 9, 2157, 1091, 7020, 618, 7252, 3213]


In [26]:
proteins.tax_id.value_counts()

2951.0       5161
28450.0      3739
2026739.0    3047
8187.0       2750
282301.0     2711
             ... 
1567104.0       1
178900.0        1
1567484.0       1
240167.0        1
1618426.0       1
Name: tax_id, Length: 41235, dtype: int64

In [28]:
proteins["taxon_count"] = proteins.groupby("tax_id")["n_complete_go_annotations"].transform("size")

In [8]:
sample_proteins = proteins[proteins['seq_len'] < 200].sample(5000).copy()
sample_proteins['seq_token_ids'] = sample_proteins['seq'].apply(sp.encode_as_ids)
sample_proteins['seq_n_tokens'] = sample_proteins['seq_token_ids'].apply(len)
sample_proteins.head()

,tax_id,uniprot_name,seq,n_complete_go_annotations,complete_go_annotation_indices,seq_len,seq_token_ids,seq_n_tokens
302093,9606.0,Q9NYY6_HUMAN,GMQKCSHIPGRRELRMPCTLGWEAYTQPQAEGVGAARNACINWNQY...,0,[],100,"[0, 8346, 6522, 4027, 991, 1176, 12230, 47, 75...",34
323931,143630.0,A0A402EGU4_9SAUR,QNKGKDINTIKSLRVLRVLRPLKTIKRLPKLKAVFDCVVNSLKNVL...,0,[],85,"[0, 3655, 2204, 2194, 1626, 6809, 7528, 132, 1...",28
890102,61819.0,A0A3Q0SJF8_AMPCI,MESFPLSRASSRNMERDWENGSIASSITSNEYTGPKLFKEPSSKSN...,0,[],169,"[0, 2211, 4652, 12997, 232, 2777, 6351, 9626, ...",57
427780,55661.0,A0A091G8V2_9AVES,MCPRPPNIANGLHSGQSLNKFSRGATVYYSCKDGYQLLGNISISCS...,0,[],175,"[0, 429, 8608, 136, 969, 2365, 1611, 2887, 189...",60
546797,62062.0,A0A4W5LM95_9TELE,ELDLSNNDLKDSGVKLLSAGLGNPHCKLETLRLSGCLVTEEGCASL...,0,[],85,"[0, 1018, 3, 1713, 599, 13094, 8279, 825, 1989...",30


In [37]:
sample_proteins.to_csv('tokenized_sample.csv',index=False)

In [9]:
display(sample_proteins['seq_len'].describe())
display(sample_proteins['seq_n_tokens'].describe())


count    1000.000000
mean      114.417000
std        46.132931
min        11.000000
25%        79.000000
50%       116.000000
75%       153.000000
max       199.000000
Name: seq_len, dtype: float64

count    1000.00000
mean       38.42600
std        15.04037
min         4.00000
25%        27.00000
50%        39.00000
75%        51.00000
max        69.00000
Name: seq_n_tokens, dtype: float64

# Let's BERT!

In [10]:
from tensorflow import keras
from bert import BertModelLayer

from tensorflow.keras.layers import Dense,Input,Flatten,concatenate,Dropout,Lambda
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K

C:\Users\Dan Ofer\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Dan Ofer\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Dan Ofer\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Dan Ofer\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWa

In [ ]:
### https://github.com/ypeleg/CAFA/blob/master/prot_bert.py

lr = 2e-5
weight_decay = 0.001
nb_epochs=10
bsz = 5
maxlen=166

# decay_steps = int(nb_epochs*sample_proteins.shape[0]/bsz)
# warmup_steps = int(0.1*decay_steps)

# class AdamWarmup1(keras.optimizers.Optimizer):
#     def __init__(self, decay_steps, warmup_steps, min_lr=0.0,
#                  lr=0.001, beta_1=0.9, beta_2=0.999,
#                  epsilon=None, kernel_weight_decay=0., bias_weight_decay=0.,
#                  amsgrad=False, **kwargs):
#         super(AdamWarmup1, self).__init__(**kwargs, name = 'something')
#         with K.name_scope(self.__class__.__name__):
#             self.decay_steps = K.variable(decay_steps, name='decay_steps')
#             self.warmup_steps = K.variable(warmup_steps, name='warmup_steps')
#             self.min_lr = K.variable(min_lr, name='min_lr')
#             self.iterations = K.variable(0, dtype='int64', name='iterations')
#             self.lr = K.variable(lr, name='lr')
#             print(self.lr) # XXX
#             self.beta_1 = K.variable(beta_1, name='beta_1')
#             self.beta_2 = K.variable(beta_2, name='beta_2')
#             self.kernel_weight_decay = K.variable(kernel_weight_decay, name='kernel_weight_decay')
#             self.bias_weight_decay = K.variable(bias_weight_decay, name='bias_weight_decay')
#         if epsilon is None:
#             epsilon = K.epsilon()
#         self.epsilon = epsilon
#         self.initial_kernel_weight_decay = kernel_weight_decay
#         self.initial_bias_weight_decay = bias_weight_decay
#         self.amsgrad = amsgrad

#     def get_updates(self, loss, params):
#         grads = self.get_gradients(loss, params)
#         self.updates = [K.update_add(self.iterations, 1)]

#         t = K.cast(self.iterations, K.floatx()) + 1

#         lr = K.switch(
#             t <= self.warmup_steps,
#             self.lr * (t / self.warmup_steps),
#             self.lr * (1.0 - K.minimum(t, self.decay_steps) / self.decay_steps),
#         )

#         lr_t = lr * (K.sqrt(1. - K.pow(self.beta_2, t)) /
#                      (1. - K.pow(self.beta_1, t)))

#         ms = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
#         vs = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
#         if self.amsgrad:
#             vhats = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
#         else:
#             vhats = [K.zeros(1) for _ in params]
#         self.weights = [self.iterations] + ms + vs + vhats

#         for p, g, m, v, vhat in zip(params, grads, ms, vs, vhats):
#             m_t = (self.beta_1 * m) + (1. - self.beta_1) * g
#             v_t = (self.beta_2 * v) + (1. - self.beta_2) * K.square(g)
#             if self.amsgrad:
#                 vhat_t = K.maximum(vhat, v_t)
#                 p_t = m_t / (K.sqrt(vhat_t) + self.epsilon)
#                 self.updates.append(K.update(vhat, vhat_t))
#             else:
#                 p_t = m_t / (K.sqrt(v_t) + self.epsilon)

#             if 'bias' in p.name or 'Norm' in p.name:
#                 if self.initial_bias_weight_decay > 0.0:
#                     p_t += self.bias_weight_decay * p
#             else:
#                 if self.initial_kernel_weight_decay > 0.0:
#                     p_t += self.kernel_weight_decay * p
#             p_t = p - lr_t * p_t

#             self.updates.append(K.update(m, m_t))
#             self.updates.append(K.update(v, v_t))
#             new_p = p_t

#             if getattr(p, 'constraint', None) is not None:
#                 new_p = p.constraint(new_p)

#             self.updates.append(K.update(p, new_p))
#         return self.updates

#     def get_config(self):
#         config = {
#             'decay_steps': float(K.get_value(self.decay_steps)),
#             'warmup_steps': float(K.get_value(self.warmup_steps)),
#             'min_lr': float(K.get_value(self.min_lr)),
#             'lr': float(K.get_value(self.lr)),
#             'beta_1': float(K.get_value(self.beta_1)),
#             'beta_2': float(K.get_value(self.beta_2)),
#             'epsilon': self.epsilon,
#             'kernel_weight_decay': float(K.get_value(self.kernel_weight_decay)),
#             'bias_weight_decay': float(K.get_value(self.bias_weight_decay)),
#             'amsgrad': self.amsgrad,
#         }
#         base_config = super(AdamWarmup1, self).get_config()
#         return dict(list(base_config.items()) + list(config.items()))


# # As the Extract layer extracts only the first token where "['CLS']" used to be, we just take the layer and connect to the single neuron output.


# adamwarm = AdamWarmup1(lr=lr,decay_steps = decay_steps, warmup_steps = warmup_steps,kernel_weight_decay = weight_decay)

In [15]:
l_bert = BertModelLayer(**BertModelLayer.Params(
  vocab_size               = 16000,        # embedding params
  use_token_type           = True,
  use_position_embeddings  = True,
  token_type_vocab_size    = 2,
    
  num_heads = 12,
  num_layers               = 12,           # transformer encoder params
  hidden_size              = 768,
  hidden_dropout           = 0.1,
  intermediate_size        = 4*768,
  intermediate_activation  = "gelu",

  adapter_size             = None,         # see arXiv:1902.00751 (adapter-BERT)

  shared_layer             = False,        # True for ALBERT (arXiv:1909.11942)
  embedding_size           = None,         # None for BERT, wordpiece embedding size for ALBERT

  name                     = "bert"        # any other Keras layer params
))

max_seq_len = 166

In [16]:
# TODO Need to use proper tokens!
PAD_TOKEN = 0
MASK_TOKEN = 1

def token_ids_to_fixed_size_sentence(seq_token_ids):
    global max_seq_len
    return np.concatenate([np.array(seq_token_ids, dtype = np.int32), \
            PAD_TOKEN * np.ones(max_seq_len - len(seq_token_ids), dtype = np.int32)])

data_tokens = np.array(sample_proteins['seq_token_ids'].apply(token_ids_to_fixed_size_sentence)\
        .tolist())
print(data_tokens.shape, data_tokens.dtype)
print(data_tokens)

(1000, 166) int32
[[   0 8346 6522 ...    0    0    0]
 [   0 3655 2204 ...    0    0    0]
 [   0 2211 4652 ...    0    0    0]
 ...
 [   0  310 3342 ...    0    0    0]
 [   0 6456 2631 ...    0    0    0]
 [   0 2343  955 ...    0    0    0]]


In [34]:
# TODO Decide what it should be
mask_freq = 0.15

data_mask = np.zeros_like(data_tokens, dtype = np.int8).flatten()
data_mask[:int((1 - mask_freq) * data_mask.size)] = 1
np.random.shuffle(data_mask)
data_mask = data_mask.reshape(data_tokens.shape)

masked_data = np.where(data_mask, data_tokens, MASK_TOKEN)

# See https://stackoverflow.com/questions/36960320/convert-a-2d-matrix-to-a-3d-one-hot-matrix-numpy/36960495
data_tokens_1he = (np.arange(l_bert.params['vocab_size']) == data_tokens[...,None]-1).astype(np.int8)
print(data_tokens_1he.shape)

(1000, 166, 16000)


In [18]:
l_input_ids = keras.layers.Input(shape=(max_seq_len,), dtype='int32')
bert_output = l_bert(l_input_ids) # output: [batch_size, max_seq_len, hidden_size]
guessed_tokens = keras.layers.Dense(l_bert.params['vocab_size'])(bert_output)
model = keras.Model(inputs=l_input_ids, outputs=guessed_tokens)
model.build(input_shape=(None, max_seq_len))


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [24]:
from  tensorflow.keras.optimizers import Adam
adm = Adam(lr=0.02, amsgrad=True)
model.compile(optimizer = adm, loss = 'sparse_categorical_crossentropy')

In [35]:
model.fit(masked_data, data_tokens_1he, batch_size = bsz, epochs = nb_epochs)

Epoch 1/10


InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument: logits and labels must have the same first dimension, got logits shape [830,16000] and labels shape [13280000]
	 [[{{node loss/dense_loss/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits}}]]
	 [[loss/mul/_2479]]
  (1) Invalid argument: logits and labels must have the same first dimension, got logits shape [830,16000] and labels shape [13280000]
	 [[{{node loss/dense_loss/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits}}]]
0 successful operations.
0 derived errors ignored.

In [ ]:
x = masked_data[0, :]
y_true = data_tokens[0, :]
y_pred = model.predict(x.reshape(1, -1)).argmax(axis = -1)[0, :]
print('x:')
print(x)
print('y_true:')
print(y_true)
print('y_pred:')
print(y_pred)